In [2]:
import random
random.sample(range(1,50),49)#生成49个1-50范围内的数组成的数列

[15,
 18,
 4,
 32,
 24,
 31,
 44,
 8,
 5,
 16,
 42,
 26,
 17,
 12,
 35,
 37,
 36,
 29,
 9,
 47,
 48,
 41,
 19,
 27,
 34,
 45,
 2,
 3,
 11,
 43,
 1,
 13,
 46,
 39,
 6,
 40,
 38,
 25,
 22,
 28,
 7,
 23,
 21,
 10,
 33,
 20,
 49,
 30,
 14]

似乎真的没有重合的数列？该方法来自[stackover问答网站](https://stackoverflow.com/questions/22842289/generate-n-unique-random-numbers-within-a-range)
不是特别理解其原理。

In [6]:
import random
random.sample(range(0,9),4)

[3, 2, 8, 0]

- 关于sample方法的用法：该方法自动包含强大的unique取样方法。
random.sample(population, k)
Return a k length list of **unique elements** chosen from the population sequence. Used for random sampling without replacement.

New in version 2.3.

Returns a new list containing elements from the population while leaving the original population unchanged. The resulting list is in selection order so that all sub-slices will also be valid random samples. This allows raffle winners (the sample) to be partitioned into grand prize and second place winners (the subslices).

Members of the population need not be hashable or unique. If the population contains repeats, then each occurrence is a possible selection in the sample.

To choose a sample from a range of integers, use an xrange() object as an argument. This is especially fast and space efficient for sampling from a large population: sample(xrange(10000000), 60).

In [16]:
import random

def compare_posi_value(targ_list=[],gue_list=[]):
    a=0
    posi_value_flag=0
    value_flag=0
    while a<4:
        if gue_list[a]==targ_list[a]:
            posi_value_flag+=1
            a+=1
        elif gue_list[a] in targ_list.pop(a):
            a+=1
            value_flag+=1
        else:
            a+=1
    print ("f{posi_value_flag}A{value_flag}B")

tag_list=random.sample(range(0,9),4)

while tag_list[0]==0:
    random.shuffle(tag_list)

i=0
while i<10:
    g=input("Please input a number between 1000~9999\n")
    if g.isdigit():
        compare_posi_value(tag_list,g)
        i+=1
    else:
        print("Please input a number!NO other type!\n")

Please input a number between 1000~9999
3456


TypeError: argument of type 'int' is not iterable

这里的问题可能出现在用`in`的语法有问题，即`gue_list[a]`不是字符串类型，而是个整数。
验证一下：


In [15]:
import types 
l=[1,2,3,4]
print(l[1])
l_1=input()
print(l_1[1])
type(l_1[3])



2
12345
2


str

以上结果可知，`input`得到的变量其实是`str`类型的，包括其内在的元素都是`str`，那上述的错误到底出在哪？
是否是定义的函数默认传入参数的问题？默认传入的是`str`类型的吗？
再次看` targ_list.pop(a)`中`pop`方法的使用，发现，原来`.pop`得到的是原来list的一个元素呀，即一个`int`类型的数字。而不是*该元素之后的list数据类型*
再次更改代码，如下：

In [17]:
import random

def compare_posi_value(targ_list=None,gue_list=None):
    a=0
    posi_value_flag=0
    value_flag=0
    targ2=[]
    while a<len(targ_list):
        print(targ_list)#for debug
        print(f"the 第{a}次循环")
        targ2=targ_list
        if gue_list[a]==targ_list[a]:
            posi_value_flag+=1
            a+=1
        else:
            targ2.pop(a)
            if gue_list[a] in targ2:
                a+=1
                value_flag+=1
            else:
                a+=1
    print (f"{posi_value_flag}A{value_flag}B")

tag_list=random.sample(range(0,9),4)

while tag_list[0]==0:
    random.shuffle(tag_list)

print(tag_list)# for debug
i=0
while i<10:
    g=input("Please input a number between 1000~9999\n")
    if g.isdigit():
        compare_posi_value(tag_list,g)
        i+=1
    else:
        print("Please input a number!NO other type!\n")

[8, 6, 0, 7]
Please input a number between 1000~9999
4568
[8, 6, 0, 7]
the 第0次循环
[6, 0, 7]
the 第1次循环
0A0B
Please input a number between 1000~9999
607
[6, 7]
the 第0次循环
0A0B
Please input a number between 1000~9999
fdif 


KeyboardInterrupt: 

可以看出运行了一次，之后因为`pop`就将目标list也取出了一个元素，导致目标list逐渐变短，因此，我们可以猜想，python在运行时是按照指针的形式指向某个数组的，对改变量的操作，直接指向了这个指针位置。因此，导致我的目标list逐渐变短。
请看：

In [18]:
a=[1,2,3]
b=a
b.pop(1)
print(a)

[1, 3]


因此，list之间的赋值可能是通过指针进行的， 关键问题就是：**如何保证对数组操作不影响原始数组？**
再次更改代码：

In [ ]:
import random

def compare_posi_value(targ_list=None,gue_list=None):
    a=0
    posi_value_flag=0
    value_flag=0
    while a<len(targ_list):
        #print(targ_list)#for debug
        print(f"the 第{a}次循环")
        print(gue_list[a])#
        print(targ_list[a])#
        if gue_list[a] == targ_list[a]:##如何判断都不为真，if后面的语句未执行。why？
            posi_value_flag+=1
            print(posi_value_flag,f"x={x}")
            a+=1
            print(f"posi_value_flag{posi_value_flag}")
        else:
            if gue_list[a] in [x!=targ_list[a] for x in targ_list]:
                a+=1
                value_flag+=1
                print("运行了第一个else")
            else:
                a+=1
                print("运行了最后一个else")

    print (f"{posi_value_flag}A{value_flag}B")

tag_list=random.sample(range(0,9),4)

while tag_list[0]==0:
    random.shuffle(tag_list)

print(tag_list)# for debug
i=0
while i<10:
    g=input("Please input a number between 1000~9999\n")
    if g.isdigit():
        compare_posi_value(tag_list,g)
        i+=1
    else:
        print("Please input a number!NO other type!\n")
